In [1]:
import json
import os
import numpy as np
import tiktoken as tt
encoding = tt.get_encoding("gpt2")
#Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://azureopenaids.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = "9382f3505dd74f2a9e609a79fe58d9e3"

deployment_name='gpt-4-32k' #This will correspond to the custom name you chose for your deployment when you deployed a model. 

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
list_of_objects = ['CB-4.json', 'CB-5.json', 'ESC-6.json', 'ESC-8.json', 'FD-11.json', 'FD-12.json', 'FL-16.json', 'FL-17.json', 'S-21.json', 'S-22.json']

In [3]:
prompt= """Extract invoice date, vendor address, invoice number, amount, project name, due date from the following invoice and seperate them by a new line.\n"""

'''        Text: September 23, 2022 Page 1 of 2\n
        Invoice Date: September 23, 2022\n
        Text: August 12, 2022 Page 1 of 2\n
        Invoice Date: August 12, 2022\n
        ###\n""" '''

In [4]:
object_filepath = list_of_objects[0]
for object_filepath in list_of_objects:
    if os.path.exists(object_filepath):
        with open( object_filepath, "r" ) as object_filepath_fd:
            try:
                object_json = json.load(object_filepath_fd)
            except Exception as e:
                print(f"** load failed, download updated version,...")

    rows = 180
    cols = 180
    chars = 256
    max_tokens = 256
    prompts=[]
    for page in range(len(object_json['analyzeResult']['pages'])):
        cast = np.empty( shape=[rows, cols, chars], dtype=str )
        width = object_json['analyzeResult']['pages'][page]['width']
        height = object_json['analyzeResult']['pages'][page]['height']
        #print(width,height)
        for word in object_json['analyzeResult']['pages'][page]['words']:
            vertices = word[ 'polygon' ]
            col = int( vertices[0]/ width*cols )
            row = int( vertices[1] / height*rows )
            #print(word[ 'content' ],vertices[0],vertices[1],col,row)
            for i, c in enumerate( word[ 'content' ] ):
                cast[row][col][i] = c
                if i >= chars:
                    break
                

                    #prompt = 'List all of the charges from the following invoice:\n\n'
                #prompt = 'Summarize the following invoice:\n\n'
            #prompt = 'Act as Invoice Parser:\n Enumerate the vendor name and address, the invoice number and amount, the property name and the project name from the following invoice:\n'
        #prompt = 'Enumerate the vendor name and address, the invoice number and amount, the property name and the project name, meter number and usage from the following invoice:\n'
        prompt= """Extract invoice date, vendor address, invoice number, amount, project name, due date from the following invoice and seperate them by a new line.\n"""
        estimate = prompt
        for row in range(rows):
            xline = ''
            for col in range(cols):
                xword = ''.join( cast[row][col] )
                xline = f'{xline}{xword} '
                if len(xword) > 0:
                    estimate = f'{estimate} {xword}'
            prompt = f'{prompt}{xline}\n'
        prompts.append(estimate)
        #print('Page ',page,'\n\n\n\n\n\n\n\n\n')
        #print(prompt)

        n_tokens = n_tokens=len(encoding.encode(estimate))
        if (n_tokens*2.6) + max_tokens > 8192:
            #print( f'** prompt tokens too large, {n_tokens*2.6} ({n_tokens}) plus {max_tokens} is greater than 4096' )
            break
        #print( f'   tokens={(n_tokens*2.6)+max_tokens} ({n_tokens})\n' )
        
        for pagenumber,prompt in enumerate(prompts):
            print('\n\nFile Name:',object_filepath)
            print('Page Number:',pagenumber+1)
            #print('\n\n\nPrompt:',prompt)
            response = openai.ChatCompletion.create(engine="gpt-4-32k", 
                                                messages = [{"role":"system","content":"You are an AI assistant that helps people find information."},{"role":"user","content":f"{prompt}"}],
                                                temperature=0.7, 
                                                max_tokens=800, 
                                                top_p=0.95,
                                                frequency_penalty=0,
                                                presence_penalty=0, 
                                                stop=None)
            print(response['choices'][0]['message']['content'])



File Name: CB-4.json
Page Number: 1
Invoice Date: 10/22/2021
Vendor Address: Not available
Invoice Number: 223560
Amount: $224.00
Project Name: 1001 Office Towers- East Tower
Due Date: Not available


File Name: CB-5.json
Page Number: 1
Invoice Date: 10/28/2021
Vendor Address: BELL MyBuildingPermit.com, a service of eCityGov.net CITY, WASHINGTON
Invoice Number: 224452
Amount: $224.00
Project Name: 1001 Office Towers- West Garage
Due Date: Not provided


File Name: ESC-6.json
Page Number: 1
Invoice Date: 3/4/2022
Vendor Address: ECS MID-ATLANTIC, LLC 14030 THUNDERBOLT PLACE, SUITE 500 CHANTILLY, VA 20151
Invoice Number: 955848
Amount: $9,241.50
Project Name: HUB Warehouse #2 Label Lane Hagerstown, MD 21740 Washington County
Due Date: DUE UPON RECEIPT


File Name: ESC-6.json
Page Number: 1
Invoice Date: 3/4/2022
Vendor Address: ECS MID-ATLANTIC, LLC 14030 THUNDERBOLT PLACE, SUITE 500 CHANTILLY, VA 20151
Invoice Number: 955848
Amount: $9,241.50
Project Name: HUB Warehouse #2 Label Lane 